In [1]:
import sys
import os
import numpy as np

from codingTools import *

In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0.raw" #int16
originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics = statistics(original_image)
entropy = statistics.compute_entropy()
print("order zero entropy of the image:", entropy)

print('Max value:', statistics.max_value)
print('Min value:', statistics.min_value)
print('Dynamic range:', statistics.dynamic_range)




order zero entropy of the image: 7.426142695578908
Max value: 255
Min value: 0
Dynamic range: 255


In [4]:
## input bitstream ##
inputbit = binary_file_input('bitpacket.txt')
inputbit.open_binay_file_input()

inputbit.read_value(10)


open ok


3

In [9]:
## output bitstream ##
outputbit = binary_file_output('test.txt')

value = 999999999
nBits = utils.needed_bits(value)
print('num of bits:', nBits)

outputbit.open_binay_file_output()
outputbit.write_value(value, nBits)
outputbit.close_binary_file_output()

print('size file:',outputbit.size_of_file())

num of bits: 30
open ok
size file: 4
